# MaRINeR Usage Demo

[[Project Page](https://boelukas.github.io/mariner/)]
<!-- [[Paper]()] -->
<!-- [[Video]()] -->
[[GitHub](https://github.com/boelukas/mariner)]

This notebook contains a demo of MaRINeR and covers project setup, training, evaluation and prediction of the model. A GPU runtime is recommended for optimal performance.


## Project Setup

In [3]:
!unzip mariner.zip
%cd mariner

Archive:  mariner.zip
   creating: mariner/.git/
   creating: mariner/.git/branches/
  inflating: mariner/.git/config     
  inflating: mariner/.git/description  
 extracting: mariner/.git/HEAD       
   creating: mariner/.git/hooks/
  inflating: mariner/.git/hooks/applypatch-msg.sample  
  inflating: mariner/.git/hooks/commit-msg.sample  
  inflating: mariner/.git/hooks/fsmonitor-watchman.sample  
  inflating: mariner/.git/hooks/post-update.sample  
  inflating: mariner/.git/hooks/pre-applypatch.sample  
  inflating: mariner/.git/hooks/pre-commit.sample  
  inflating: mariner/.git/hooks/pre-merge-commit.sample  
  inflating: mariner/.git/hooks/prepare-commit-msg.sample  
  inflating: mariner/.git/hooks/pre-push.sample  
  inflating: mariner/.git/hooks/pre-rebase.sample  
  inflating: mariner/.git/hooks/pre-receive.sample  
  inflating: mariner/.git/hooks/update.sample  
  inflating: mariner/.git/index      
   creating: mariner/.git/info/
  inflating: mariner/.git/info/exclude  
   cr

### Download Demo Dataset and Pre-Trained Weights



In [4]:
!mkdir pretrained_weights
!mkdir data
!gdown 1zb90JWtX5-Si7MklJMqWn1Kwnqsi6mhF
!gdown 1VmhgXL1IFRwDlCSPZcwTt9ZsKorSknKk
!unzip demo_data.zip -d data/demo_data
!unzip mariner.zip -d pretrained_weights/

Downloading...
From (original): https://drive.google.com/uc?id=1zb90JWtX5-Si7MklJMqWn1Kwnqsi6mhF
From (redirected): https://drive.google.com/uc?id=1zb90JWtX5-Si7MklJMqWn1Kwnqsi6mhF&confirm=t&uuid=2e397437-b210-4962-99db-bed079e7ce12
To: /content/mariner/mariner.zip
100% 176M/176M [00:04<00:00, 38.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VmhgXL1IFRwDlCSPZcwTt9ZsKorSknKk
To: /content/mariner/demo_data.zip
100% 6.87M/6.87M [00:00<00:00, 41.8MB/s]
Archive:  demo_data.zip
  inflating: data/demo_data/gt/CAB_00046_gt.png  
  inflating: data/demo_data/gt/CAB_00082_gt.png  
  inflating: data/demo_data/gt/CAB_00172_gt.png  
  inflating: data/demo_data/gt/CAB_00282_gt.png  
  inflating: data/demo_data/gt/HGE_00169_gt.png  
  inflating: data/demo_data/gt/HGE_00244_gt.png  
  inflating: data/demo_data/gt/LIN_00019_gt.png  
  inflating: data/demo_data/gt/LIN_00055_gt.png  
  inflating: data/demo_data/gt/LIN_00148_gt.png  
  inflating: data/demo_data/gt/LIN_00330_gt.png  
  inflati

### Install Dependencies

In [6]:
!pip install torchvision==0.16.0 tensorboard===2.17.0 lightning==2.2.1 opencv-python==4.10.0.84 jsonargparse==4.31.0 erqa==1.1.2 lpips==0.1.4 tabulate==0.9.0 rich==13.7.1 numpy==1.26.4 jsonargparse[signatures]>=4.26.1

## Predict

In [5]:
!python mariner/main.py predict \
    -c configs/MaRINeR.yml \
    --ckpt_path /content/mariner/pretrained_weights/mariner.ckpt \
    --data_dir /content/mariner/data/demo_data

2024-07-08 10:02:01.427868: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 10:02:01.427926: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 10:02:01.536116: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-08 10:02:01.747124: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-08 10:02:03.834869: W tensorflow/compiler/tf2

### Visualize Results

In [6]:
!pip install ipympl --quiet
%matplotlib widget
from google.colab import output
output.enable_custom_widget_manager()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.2 MB/s eta 0:00:00


In [7]:

import matplotlib.pyplot as plt
import os
import ipywidgets as widgets
from PIL import Image

ref_dir = "/content/mariner/data/demo_data/ref"
input_dir = "/content/mariner/data/demo_data/input"
out_dir = "/content/mariner/data/demo_data/out"
gt_dir = "/content/mariner/data/demo_data/gt"

input_files = sorted(os.listdir(input_dir))
out_files = sorted(os.listdir(out_dir))
gt_files = sorted(os.listdir(gt_dir))
ref_files = sorted(os.listdir(ref_dir))
scale_factor = 1.0

def update_fig(fig, ax, input_img, gt_img, ref_img, out_img):
    img_width, img_height = input_img.size

    fig.set_size_inches(scale_factor*img_width * 3 / 100, (scale_factor*img_height / 100) + 0.5)
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0.05, hspace=0.05)

    for ax in axs:
      ax.clear()
      ax.axis('off')

    axs[0].imshow(ref_img)
    axs[0].set_title("Reference")
    axs[1].imshow(input_img)
    axs[1].set_title("Input")
    axs[2].imshow(out_img)
    axs[2].set_title("Output")
    axs[3].imshow(gt_img)
    axs[3].set_title("Ground Truth")

    fig.canvas.draw_idle()


def update_scale(change):
    global scale_factor
    scale_factor = change.new
    update_fig(fig, axs, Image.open(os.path.join(input_dir, input_files[slider.value])),
               Image.open(os.path.join(gt_dir, gt_files[slider.value])),
               Image.open(os.path.join(ref_dir, ref_files[slider.value])),
               Image.open(os.path.join(out_dir, out_files[slider.value])))


def update(change):
    new_input_img = Image.open(os.path.join(input_dir, input_files[change.new]))
    new_gt_img = Image.open(os.path.join(gt_dir, gt_files[change.new]))
    new_out_img = Image.open(os.path.join(out_dir, out_files[change.new]))

    new_ref_img = Image.open(os.path.join(ref_dir, ref_files[change.new]))

    update_fig(fig, axs, new_input_img, new_gt_img, new_ref_img, new_out_img)

with plt.ioff():
    fig, axs = plt.subplots(1, 4)
    for ax in axs:
        ax.axis('off')
        input_image = Image.open(os.path.join(input_dir, input_files[0]))
        gt_image = Image.open(os.path.join(gt_dir, gt_files[0]))
        out_image = Image.open(os.path.join(out_dir, out_files[0]))

        ref_image = Image.open(os.path.join(ref_dir, ref_files[0]))

        update_fig(fig, axs, input_image, gt_image, ref_image, out_image)

slider = widgets.IntSlider(value=0, min=0, max=len(input_files)-1, description='Image Index:')
slider.observe(update, names='value')

scale_slider = widgets.FloatSlider(value=1.0, min=0, max=10.0, description='Scale:')
scale_slider.observe(update_scale, names='value')
scale_text = widgets.FloatText(value=1.0, description='Scale Factor:', layout=widgets.Layout(width='150px'))
scale_text.observe(update_scale, names='value')

widgets.VBox([fig.canvas, slider, scale_slider])

## Evaluate

### Download Evaluation Datasets

In [7]:
# https://drive.google.com/file/d/1fkajRAyxsaOsCPxZLDU1iUMo8BYZNGej/view?usp=drive_link
!gdown 1fkajRAyxsaOsCPxZLDU1iUMo8BYZNGej
!unzip test_data.zip -d data/test_data

Streaming output truncated to the last 5000 lines.
  inflating: data/test_data/CAB_ref_gt/input/00293.png  
  inflating: data/test_data/CAB_ref_gt/input/00294.png  
  inflating: data/test_data/CAB_ref_gt/input/00295.png  
  inflating: data/test_data/CAB_ref_gt/input/00296.png  
  inflating: data/test_data/CAB_ref_gt/input/00297.png  
  inflating: data/test_data/CAB_ref_gt/input/00298.png  
  inflating: data/test_data/CAB_ref_gt/input/00299.png  
  inflating: data/test_data/CAB_ref_gt/input/00300.png  
  inflating: data/test_data/CAB_ref_gt/input/00301.png  
  inflating: data/test_data/CAB_ref_gt/input/00302.png  
  inflating: data/test_data/CAB_ref_gt/input/00303.png  
  inflating: data/test_data/CAB_ref_gt/input/00304.png  
  inflating: data/test_data/CAB_ref_gt/input/00305.png  
  inflating: data/test_data/CAB_ref_gt/input/00306.png  
  inflating: data/test_data/CAB_ref_gt/input/00307.png  
  inflating: data/test_data/CAB_ref_gt/input/00308.png  
  inflating: data/test_data/CAB_ref_g

### Predict Test Data


In [10]:
dataset = "NeRF" # @param ["CAB_ref_gt", "CAB_ref_lvl_1", "LIN_ref_lvl_1", "HGE_ref_lvl_1", "IBRnet", "12SCENES_apt_1_living_ref_lvl_10", "NeRF"] {allow-input: false}
!python mariner/main.py predict \
    -c configs/MaRINeR.yml \
    --ckpt_path /content/mariner/pretrained_weights/mariner.ckpt \
    --data_dir /content/mariner/data/test_data/{dataset}/


2024-07-08 12:46:21.974865: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 12:46:21.974967: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 12:46:21.977024: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-08 12:46:21.987421: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-08 12:46:23.664059: W tensorflow/compiler/tf2

### Evaluate Test Data

In [13]:
!python scripts/eval_metrics.py \
    --images /content/mariner/data/test_data/{dataset}/out \
            /content/mariner/data/test_data/{dataset}/gt

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth
         PSNR      SSIM      ERQA    LPIPS_ALEX
---  --------  --------  --------  ------------
AVG  20.4479   0.591761  0.700466     0.167444
STD   2.16737  0.14705   0.061346     0.0878363


## Training

### Download Training Dataset

In [14]:
# https://drive.google.com/file/d/1x9Q6np6VklEthr5f3Ne15pUzfcc7Megk/view?usp=drive_link
!gdown 1x9Q6np6VklEthr5f3Ne15pUzfcc7Megk
!unzip train_data.zip -d data/train_data

Streaming output truncated to the last 5000 lines.
  inflating: data/train_data/CAB_merged_LIN/train/ref/33745_00.png  
  inflating: data/train_data/CAB_merged_LIN/train/ref/33745_01.png  
  inflating: data/train_data/CAB_merged_LIN/train/ref/33745_02.png  
  inflating: data/train_data/CAB_merged_LIN/train/ref/33745_03.png  
  inflating: data/train_data/CAB_merged_LIN/train/ref/33746_00.png  
  inflating: data/train_data/CAB_merged_LIN/train/ref/33746_01.png  
  inflating: data/train_data/CAB_merged_LIN/train/ref/33746_02.png  
  inflating: data/train_data/CAB_merged_LIN/train/ref/33746_03.png  
  inflating: data/train_data/CAB_merged_LIN/train/ref/33747_00.png  
  inflating: data/train_data/CAB_merged_LIN/train/ref/33747_01.png  
  inflating: data/train_data/CAB_merged_LIN/train/ref/33747_02.png  
  inflating: data/train_data/CAB_merged_LIN/train/ref/33747_03.png  
  inflating: data/train_data/CAB_merged_LIN/train/ref/33748_00.png  
  inflating: data/train_data/CAB_merged_LIN/train/re

### Train

Adjust the `batch_size` according to the runtime. To train with the default parameters `batch_size = 9`, 32 GB VRAM are needed.

In [18]:
batch_size = 2 # @param {type:"number"}

!python mariner/main.py fit \
     -c configs/MaRINeR.yml \
     --train_data_dir /content/mariner/data/train_data/CAB_merged_LIN/train \
     --test_data_dir //content/mariner/data/train_data/CAB_merged_LIN/test \
     --batch_size {batch_size}

2024-07-08 13:17:46.384709: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 13:17:46.384810: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 13:17:46.502019: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-08 13:17:46.724322: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-08 13:17:49.371692: W tensorflow/compiler/tf2